In [27]:
import pandas as pd
df = pd.read_csv('BTCUSDT.csv')

In [28]:
print(df['timestamp'].dtype)

object


In [29]:
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')  # Use 'coerce' to handle invalid parsing

In [30]:
df['year'] = df['timestamp'].dt.year.astype(int)
df['month'] = df['timestamp'].dt.month.astype(int)
df['day'] = df['timestamp'].dt.day.astype(int)
df['hour'] = df['timestamp'].dt.hour.astype(int)
df['minute'] = df['timestamp'].dt.minute.astype(int)
df['day_name'] = df['timestamp'].dt.day_name()

In [31]:
df['max'] = df[['open', 'high', 'low', 'close']].max(axis=1)
df['min'] = df[['open', 'high', 'low', 'close']].min(axis=1)

In [32]:
df = df.iloc[1:,:]

In [33]:
df

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore,year,month,day,hour,minute,day_name,max,min
1,2017-08-17 04:01:00,4261.48,4261.48,4261.48,4261.48,0.000000,2017-08-17 04:01:59.999,0.000000,0,0.000000,0.000000,0,2017,8,17,4,1,Thursday,4261.48,4261.48
2,2017-08-17 04:02:00,4280.56,4280.56,4280.56,4280.56,0.261074,2017-08-17 04:02:59.999,1117.542921,2,0.261074,1117.542921,0,2017,8,17,4,2,Thursday,4280.56,4280.56
3,2017-08-17 04:03:00,4261.48,4261.48,4261.48,4261.48,0.012008,2017-08-17 04:03:59.999,51.171852,3,0.012008,51.171852,0,2017,8,17,4,3,Thursday,4261.48,4261.48
4,2017-08-17 04:04:00,4261.48,4261.48,4261.48,4261.48,0.140796,2017-08-17 04:04:59.999,599.999338,1,0.140796,599.999338,0,2017,8,17,4,4,Thursday,4261.48,4261.48
5,2017-08-17 04:05:00,4261.48,4261.48,4261.48,4261.48,0.000000,2017-08-17 04:05:59.999,0.000000,0,0.000000,0.000000,0,2017,8,17,4,5,Thursday,4261.48,4261.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3745385,2024-10-06 02:56:00,61959.99,61960.00,61959.99,61960.00,2.253500,2024-10-06 02:56:59.999000,139626.847230,116,0.976510,60504.559600,0,2024,10,6,2,56,Sunday,61960.00,61959.99
3745386,2024-10-06 02:57:00,61959.99,61970.00,61959.99,61969.99,10.088480,2024-10-06 02:57:59.999000,625088.914245,819,9.735270,603203.997688,0,2024,10,6,2,57,Sunday,61970.00,61959.99
3745387,2024-10-06 02:58:00,61970.00,61970.00,61969.99,61969.99,2.524880,2024-10-06 02:58:59.999000,156466.811885,144,2.353370,145838.338900,0,2024,10,6,2,58,Sunday,61970.00,61969.99
3745388,2024-10-06 02:59:00,61970.00,61970.00,61966.00,61966.01,0.958310,2024-10-06 02:59:59.999000,59386.125299,290,0.057220,3545.922841,0,2024,10,6,2,59,Sunday,61970.00,61966.00


In [34]:
dfs = []

for _, group in df.groupby(['year','month', 'day']):
    for i in range(0, len(group), 15):
        dfs.append(group.iloc[i:i+15])

df_15m = pd.DataFrame()

for df in dfs:
    ohlc_data = pd.DataFrame({
    'high' : [df['max'].max()],
    'low' : [df['min'].min()],
    'open' : [df.iloc[0].open],
    'close' : [df.iloc[-1].close],
    'timestamp': [df.iloc[0].timestamp],
    })

    ohlc_data.index = pd.to_datetime([f"{int(df.iloc[-1].year)}-{int(df.iloc[-1].month)}-{int(df.iloc[-1].day)}"])
    df_15m = pd.concat([df_15m, ohlc_data])

In [35]:
df_15m['year'] = df_15m['timestamp'].dt.year.astype(int)
df_15m['month'] = df_15m['timestamp'].dt.month.astype(int)
df_15m['day'] = df_15m['timestamp'].dt.day.astype(int)
df_15m['hour'] = df_15m['timestamp'].dt.hour.astype(int)
df_15m['minute'] = df_15m['timestamp'].dt.minute.astype(int)
df_15m['day_name'] = df_15m['timestamp'].dt.day_name()
df_15m['max'] = df_15m[['open', 'high', 'low', 'close']].max(axis=1)
df_15m['min'] = df_15m[['open', 'high', 'low', 'close']].min(axis=1)
df_15m['difference'] = df_15m['max'] - df_15m['min']
df_15m['candle_body'] = df_15m['open'] - df_15m['close']
df_15m['high_shadow'] = df_15m['high'] - df_15m[['open', 'close']].max(axis=1)
df_15m['low_shadow'] = df_15m['low'] - df_15m[['open', 'close']].min(axis=1)
df_15m['low-open'] = df_15m['low'] - df_15m['open']
df_15m['high-open'] = df_15m['high'] - df_15m['open']
df_15m['trend'] = df_15m['candle_body'].apply(lambda x: "UP" if x < 0 else "ZERO" if x == 0 else "DOWN")
df_15m['7_EMA'] = df_15m['close'].ewm(span=7, adjust=False).mean()
df_15m['15_EMA'] = df_15m['close'].ewm(span=15, adjust=False).mean()
df_15m['50_EMA'] = df_15m['close'].ewm(span=50, adjust=False).mean()
df_15m['200_EMA'] = df_15m['close'].ewm(span=200, adjust=False).mean()

In [36]:
df_15m

,high,low,open,close,timestamp,year,month,day,hour,minute,...,candle_body,high_shadow,low_shadow,low-open,high-open,trend,7_EMA,15_EMA,50_EMA,200_EMA
2017-08-17,4280.56,4261.48,4261.48,4261.48,2017-08-17 04:01:00,2017,8,17,4,1,...,0.00,19.08,0.00,0.00,19.08,ZERO,4261.480000,4261.480000,4261.480000,4261.480000
2017-08-17,4280.56,4261.32,4261.48,4280.00,2017-08-17 04:16:00,2017,8,17,4,16,...,-18.52,0.56,-0.16,-0.16,19.08,UP,4266.110000,4263.795000,4262.206275,4261.664279
2017-08-17,4310.07,4267.99,4274.67,4310.07,2017-08-17 04:31:00,2017,8,17,4,31,...,-35.40,0.00,-6.68,-6.68,35.40,UP,4277.100000,4269.579375,4264.083283,4262.145928
2017-08-17,4313.62,4291.37,4310.07,4308.83,2017-08-17 04:46:00,2017,8,17,4,46,...,1.24,3.55,-17.46,-18.70,3.55,DOWN,4285.032500,4274.485703,4265.838057,4262.610446
2017-08-17,4328.69,4304.31,4308.83,4320.00,2017-08-17 05:01:00,2017,8,17,5,1,...,-11.17,8.69,-4.52,-4.52,19.86,UP,4293.774375,4280.174990,4267.962054,4263.181486
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-06,62115.99,61982.86,62081.01,61996.01,2024-10-06 02:00:00,2024,10,6,2,0,...,85.00,34.98,-13.15,-98.15,34.98,DOWN,62024.481914,62001.300099,62000.233381,61860.990844
2024-10-06,62044.00,61996.00,61996.01,62040.01,2024-10-06 02:15:00,2024,10,6,2,15,...,-44.00,3.99,-0.01,-0.01,47.99,UP,62028.363936,62006.138836,62001.793248,61862.772129
2024-10-06,62040.02,61940.11,62040.02,61957.87,2024-10-06 02:30:00,2024,10,6,2,30,...,82.15,0.00,-17.76,-99.91,0.00,DOWN,62010.740452,62000.105232,62000.070768,61863.718376
2024-10-06,61991.00,61956.05,61957.87,61966.01,2024-10-06 02:45:00,2024,10,6,2,45,...,-8.14,24.99,-1.82,-1.82,33.13,UP,61999.557839,61995.843328,61998.735051,61864.736204


In [37]:
df_15m_2023_and_2024 = df_15m[df_15m['year'].isin([2023,2024])]

In [38]:
df_15m_2023_and_2024.to_csv('BTC_USD_15_MINUTE_DF.csv',index=True)